In [328]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from mpl_toolkits import mplot3d
import pickle # convert model into bytestream

pd.options.mode.chained_assignment = None 

In [329]:
ts_data = pd.read_csv("D:\This Project\Credit Spend\daily.csv")

In [330]:
ts_data.head()

,Date,Mode,Category,Subcategory,Note,Amount,Income/Expense,Currency
0,20/09/2018 12:04:08,Cash,Transportation,Train,2 Place 5 to Place 0,30.0,Expense,INR
1,20/09/2018 12:03:15,Cash,Food,snacks,Idli medu Vada mix 2 plates,60.0,Expense,INR
2,19/09/2018,Saving Bank account 1,subscription,Netflix,1 month subscription,199.0,Expense,INR
3,17/09/2018 23:41:17,Saving Bank account 1,subscription,Mobile Service Provider,Data booster pack,19.0,Expense,INR
4,16/09/2018 17:15:08,Cash,Festivals,Ganesh Pujan,Ganesh idol,251.0,Expense,INR


In [331]:
ts_data.sample()

,Date,Mode,Category,Subcategory,Note,Amount,Income/Expense,Currency
229,24/06/2018 13:31:06,Cash,Transportation,Train,2 Place 5 to Place 0,30.0,Expense,INR


In [332]:
ts_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            2461 non-null   object 
 1   Mode            2461 non-null   object 
 2   Category        2461 non-null   object 
 3   Subcategory     1826 non-null   object 
 4   Note            1940 non-null   object 
 5   Amount          2461 non-null   float64
 6   Income/Expense  2461 non-null   object 
 7   Currency        2461 non-null   object 
dtypes: float64(1), object(7)
memory usage: 153.9+ KB


In [333]:
ts_data.dropna( inplace = True )


In [334]:
st = ts_data.iloc[0][0]
st = st.split(' ')[0]
st = st.split('/')[2]
st

'2018'

In [335]:
len( ts_data['Date'])

1563

In [336]:

def extract_month ( data , col ) :
    months = []
    rows = len(data[col]) 
    for i in range(rows ) :

        st = data.iloc[i][col]
        month_= st.split(' ')[0]
        month = month_.split('/')[1]
        months.append( int(month) )
    
    return months 

Month = extract_month( ts_data , 'Date')

ts_data.drop( 'Date' , axis=1 , inplace= True )

In [337]:
ts_data['Months'] = Month

In [338]:
''' Cheking Imbalance '''
ts_data['Months'].value_counts()

Months
3     188
5     173
1     149
7     148
2     147
8     132
11    125
4     114
6     112
12    112
9      94
10     69
Name: count, dtype: int64

In [339]:
'''  Sort the months column and go further then ONLY '''

'  Sort the months column and go further then ONLY '

In [340]:
''' USe feature scaling for better results , GIve 2D input lo\ike OHE'''

from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit_transform(  ts_data[['Amount' , 'Months']] )


array([[-0.17721915,  0.88544374],
       [-0.16525486,  0.88544374],
       [-0.10982036,  0.88544374],
       ...,
       [ 0.02059032, -1.16702235],
       [-0.17123701, -1.16702235],
       [-0.17562391, -1.16702235]])

In [341]:
X = pd.DataFrame(ts_data['Months'])

y = pd.DataFrame(ts_data[ 'Amount'])



In [342]:
mu = np.mean(y)
std_ = np.std(y)


In [343]:
from sklearn.mixture import GaussianMixture 

gmm = GaussianMixture(n_components= 6)
gmm.fit(X, y)

GaussianMixture(n_components=6)

In [344]:
y1,x1 = gmm.sample( n_samples=10)
''' Sampling new datapoit to OVERSAMPLE MINORITY CONTINUOIS VALUE 
SMOTE can only be sued for cat. data '''

' Sampling new datapoit to OVERSAMPLE MINORITY CONTINUOIS VALUE \nSMOTE can only be sued for cat. data '

In [345]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test  = X[:1553], X[1553:] , y[:1553] , y[1553:]

In [346]:
X_train.iloc[0].shape

(1,)

In [347]:

from tensorflow import keras 
from keras.layers import Dense  , LSTM 
# Model 

rnn = keras.models.Sequential([
    LSTM( units =50 , activation ="relu" , input_shape = (1 , 1 ) , return_sequences=True) , 
    LSTM( units= 50 , activation='relu' ), 
    # LSTM( units= 50 , activation='relu' ), 
    Dense(1 )
])

rnn.compile( loss= 'mse' , optimizer='sgd' , metrics='mse')

In [348]:
X.iloc[0]

Months    9
Name: 0, dtype: int64

In [349]:
rnn.fit( X_train , y_train  ,  epochs =200)

Epoch 1/200
49/49 [==============================] - 2s 2ms/step - loss: 6816277.0000 - mse: 6816277.0000
Epoch 2/200
49/49 [==============================] - 0s 2ms/step - loss: 6352845.0000 - mse: 6352845.0000
Epoch 3/200
49/49 [==============================] - 0s 2ms/step - loss: 6331498.0000 - mse: 6331498.0000
Epoch 4/200
49/49 [==============================] - 0s 2ms/step - loss: 6329289.5000 - mse: 6329289.5000
Epoch 5/200
49/49 [==============================] - 0s 2ms/step - loss: 6330146.5000 - mse: 6330146.5000
Epoch 6/200
49/49 [==============================] - 0s 2ms/step - loss: 6333136.5000 - mse: 6333136.5000
Epoch 7/200
49/49 [==============================] - 0s 2ms/step - loss: 6330693.0000 - mse: 6330693.0000
Epoch 8/200
49/49 [==============================] - 0s 2ms/step - loss: 6330108.0000 - mse: 6330108.0000
Epoch 9/200
49/49 [==============================] - 0s 2ms/step - loss: 6330577.5000 - mse: 6330577.5000
Epoch 10/200
49/49 [==========================

In [350]:
y_pred = rnn.predict( X_test )
from sklearn.metrics import r2_score

score = r2_score( y_test , y_pred) 
score

1/1 [==============================] - 0s 325ms/step


-6.08914604336478

In [358]:
np.set_printoptions(suppress=True, formatter={'float_kind':'{:0.2f}'.format})
''' Print in actual val , NOT scientific notation'''

' Print in actual val , NOT scientific notation'

In [360]:
y_1 = rnn.predict(X_train[:1])
val = y_1[0][0]
val=  val*std_ + mu 



1/1 [==============================] - 0s 22ms/step


In [361]:
print(f"Predicted Transcation - {val} INR")

Predicted Transcation - Amount    1.162697e+06
dtype: float64 INR
